# BasePreprocessing

<font size=5>PUBG Finish Placement Prediction kaggle 대회 Data Leagkage 문제로 인해</font>  
<font size=5>성능 개선의 기준이 될만한 지표를 만들기로 하였습니다.</font>

- 'killPlace' 컬럼의 data Leakage 문제로 전처리
- 'Id', 'groupId', 'matchId', 'matchType' objective 변수 컬럼에 대해서 전처리
- 'target' 제외 총 23개 column 대한 BaseModel 학습 및 score 계산

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('../data/train_V2.csv')
df = df.dropna()
df = df.drop(columns=['Id','groupId','matchId','matchType','killPlace'])
df.head()

,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPoints,kills,killStreaks,longestKill,...,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
0,0,0,0.00,0,0,0,1241,0,0,0.00,...,0,0.0000,0,0.00,0,0,244.80,1,1466,0.4444
1,0,0,91.47,0,0,0,0,0,0,0.00,...,0,0.0045,0,11.04,0,0,1434.00,5,0,0.6400
2,1,0,68.00,0,0,0,0,0,0,0.00,...,0,0.0000,0,0.00,0,0,161.80,2,0,0.7755
3,0,0,32.90,0,0,0,0,0,0,0.00,...,0,0.0000,0,0.00,0,0,202.70,3,0,0.1667
4,0,0,100.00,0,0,0,0,1,1,58.53,...,0,0.0000,0,0.00,0,0,49.75,2,0,0.1875


In [3]:
y = df[['winPlacePerc']]
X = df.drop(columns='winPlacePerc')
display(y)
display(X)

,winPlacePerc
0,0.4444
1,0.6400
2,0.7755
3,0.1667
4,0.1875
...,...
4446961,0.1786
4446962,0.2935
4446963,0.4815
4446964,0.8000


,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPoints,kills,killStreaks,longestKill,...,rankPoints,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints
0,0,0,0.00,0,0,0,1241,0,0,0.00,...,-1,0,0.0000,0,0.000,0,0,244.80,1,1466
1,0,0,91.47,0,0,0,0,0,0,0.00,...,1484,0,0.0045,0,11.040,0,0,1434.00,5,0
2,1,0,68.00,0,0,0,0,0,0,0.00,...,1491,0,0.0000,0,0.000,0,0,161.80,2,0
3,0,0,32.90,0,0,0,0,0,0,0.00,...,1408,0,0.0000,0,0.000,0,0,202.70,3,0
4,0,0,100.00,0,0,0,0,1,1,58.53,...,1560,0,0.0000,0,0.000,0,0,49.75,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4446961,0,0,0.00,0,0,0,1029,0,0,0.00,...,-1,0,1292.0000,0,0.000,0,0,1019.00,3,1507
4446962,0,1,44.15,0,0,0,0,0,0,0.00,...,1501,0,0.0000,0,0.000,0,0,81.70,6,0
4446963,0,0,59.06,0,0,0,0,0,0,0.00,...,1500,0,0.0000,0,2.184,0,0,788.70,4,0
4446964,0,4,180.40,1,1,2,0,2,1,98.50,...,1418,2,0.0000,0,0.000,0,0,2748.00,8,0


In [4]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

((3557572, 23), (3557572, 1), (889393, 23), (889393, 1))

# BaseModelScore

- 1. xgboostRegressor에 CrossValidation 라이브러리 및 기본적인 하이퍼파라미터 설정을 통해 MAE스코어 계산
- 2. sklearn KFold 라이브러리를 이용하여 동일한 조건으로 xgboostRegressor에 대한 CrossValidation 진행한 MAE 스코어 계산
- 3. 두번의 BaseModel CrossValidation mae 스코어를 바탕으로 성능 개선 예정

In [5]:
import xgboost
from sklearn.model_selection import KFold

In [6]:
dtrain = xgboost.DMatrix(X_train, label=y_train)
params = {'booster':'gbtree',
            'gamma':0,
            'learning_rate':0.05,
            'max_depth':5,
            'min_child_weight':1,
            'n_jobs':-1,
            'objective':'reg:squaredlogerror',
            'reg_alpha':0,
          'eval_metric' : 'mae',
         'random_state':42}

xgb_cv = xgboost.cv(
  num_boost_round=100,
  params = params,
  dtrain = dtrain,
  nfold=5,
  as_pandas = True
)

In [11]:
# xgb.cv:num_boost_round와 xgb(): n_estimater 파라미터는 같은 것이다! defalut는 document에 없지만 100입니다
xgb_cv

,train-mae-mean,train-mae-std,test-mae-mean,test-mae-std
0,0.256232,0.000037,0.256233,0.000154
1,0.244335,0.000040,0.244338,0.000160
2,0.233525,0.000040,0.233529,0.000170
3,0.223635,0.000040,0.223639,0.000177
4,0.214565,0.000040,0.214570,0.000180
...,...,...,...,...
95,0.091993,0.000035,0.092074,0.000114
96,0.091941,0.000037,0.092023,0.000112
97,0.091892,0.000035,0.091974,0.000114
98,0.091844,0.000033,0.091926,0.000115


In [6]:
from sklearn.metrics import mean_absolute_error

# eval_set 하이퍼파라미터를 사용하면 아래와 같이 결과를 출력합니다

model = xgboost.XGBRegressor(
    n_estimators=100,
    booster='gbtree',
    gamma=0,
    learning_rate=0.05,
    max_depth=5,
    min_child_weight=1,
    n_jobs=-1,
    objective='reg:squaredlogerror',
    # 주의: XGBRegressor는 sklearn mean_absolute_error 사용 (안하면 출력은 reg:squaredlogerror 값만 출력 됨)
    # xgb.cv 같은 경우 'mae'로 설정해야 됩니다.
    eval_metric=mean_absolute_error,
    reg_alpha=0,
    random_state=42)

scores = []
folds = KFold(n_splits=5)

for train_idx, val_idx in folds.split(X, y):
    X_train = X.iloc[train_idx, :]
    y_train = y.iloc[train_idx]

    X_val = X.iloc[val_idx, :]
    y_val = y.iloc[val_idx]

    model.fit(X_train, y_train, eval_set=[(X_val, y_val)])
    preds = model.predict(X_val)
    score = mean_absolute_error(y_val, preds)
    scores.append(score)

[0]	validation_0-rmsle:0.20636	validation_0-mean_absolute_error:0.25632
[1]	validation_0-rmsle:0.19694	validation_0-mean_absolute_error:0.24443
[2]	validation_0-rmsle:0.18838	validation_0-mean_absolute_error:0.23363
[3]	validation_0-rmsle:0.18059	validation_0-mean_absolute_error:0.22373
[4]	validation_0-rmsle:0.17347	validation_0-mean_absolute_error:0.21467
[5]	validation_0-rmsle:0.16695	validation_0-mean_absolute_error:0.20633
[6]	validation_0-rmsle:0.16096	validation_0-mean_absolute_error:0.19862
[7]	validation_0-rmsle:0.15545	validation_0-mean_absolute_error:0.19148
[8]	validation_0-rmsle:0.15038	validation_0-mean_absolute_error:0.18488
[9]	validation_0-rmsle:0.14569	validation_0-mean_absolute_error:0.17874
[10]	validation_0-rmsle:0.14138	validation_0-mean_absolute_error:0.17306
[11]	validation_0-rmsle:0.13740	validation_0-mean_absolute_error:0.16776
[12]	validation_0-rmsle:0.13375	validation_0-mean_absolute_error:0.16289
[13]	validation_0-rmsle:0.13037	validation_0-mean_absolute_er

[13]	validation_0-rmsle:0.13028	validation_0-mean_absolute_error:0.15819
[14]	validation_0-rmsle:0.12717	validation_0-mean_absolute_error:0.15396
[15]	validation_0-rmsle:0.12429	validation_0-mean_absolute_error:0.14999
[16]	validation_0-rmsle:0.12163	validation_0-mean_absolute_error:0.14628
[17]	validation_0-rmsle:0.11919	validation_0-mean_absolute_error:0.14284
[18]	validation_0-rmsle:0.11692	validation_0-mean_absolute_error:0.13960
[19]	validation_0-rmsle:0.11483	validation_0-mean_absolute_error:0.13661
[20]	validation_0-rmsle:0.11292	validation_0-mean_absolute_error:0.13382
[21]	validation_0-rmsle:0.11117	validation_0-mean_absolute_error:0.13123
[22]	validation_0-rmsle:0.10956	validation_0-mean_absolute_error:0.12881
[23]	validation_0-rmsle:0.10805	validation_0-mean_absolute_error:0.12651
[24]	validation_0-rmsle:0.10667	validation_0-mean_absolute_error:0.12439
[25]	validation_0-rmsle:0.10541	validation_0-mean_absolute_error:0.12243
[26]	validation_0-rmsle:0.10423	validation_0-mean_a

[26]	validation_0-rmsle:0.10422	validation_0-mean_absolute_error:0.12056
[27]	validation_0-rmsle:0.10315	validation_0-mean_absolute_error:0.11883
[28]	validation_0-rmsle:0.10216	validation_0-mean_absolute_error:0.11721
[29]	validation_0-rmsle:0.10125	validation_0-mean_absolute_error:0.11571
[30]	validation_0-rmsle:0.10041	validation_0-mean_absolute_error:0.11429
[31]	validation_0-rmsle:0.09963	validation_0-mean_absolute_error:0.11295
[32]	validation_0-rmsle:0.09892	validation_0-mean_absolute_error:0.11172
[33]	validation_0-rmsle:0.09827	validation_0-mean_absolute_error:0.11058
[34]	validation_0-rmsle:0.09766	validation_0-mean_absolute_error:0.10952
[35]	validation_0-rmsle:0.09711	validation_0-mean_absolute_error:0.10851
[36]	validation_0-rmsle:0.09659	validation_0-mean_absolute_error:0.10756
[37]	validation_0-rmsle:0.09612	validation_0-mean_absolute_error:0.10669
[38]	validation_0-rmsle:0.09568	validation_0-mean_absolute_error:0.10586
[39]	validation_0-rmsle:0.09529	validation_0-mean_a

[39]	validation_0-rmsle:0.09521	validation_0-mean_absolute_error:0.10510
[40]	validation_0-rmsle:0.09484	validation_0-mean_absolute_error:0.10438
[41]	validation_0-rmsle:0.09447	validation_0-mean_absolute_error:0.10367
[42]	validation_0-rmsle:0.09415	validation_0-mean_absolute_error:0.10304
[43]	validation_0-rmsle:0.09385	validation_0-mean_absolute_error:0.10242
[44]	validation_0-rmsle:0.09357	validation_0-mean_absolute_error:0.10184
[45]	validation_0-rmsle:0.09331	validation_0-mean_absolute_error:0.10132
[46]	validation_0-rmsle:0.09307	validation_0-mean_absolute_error:0.10081
[47]	validation_0-rmsle:0.09286	validation_0-mean_absolute_error:0.10035
[48]	validation_0-rmsle:0.09266	validation_0-mean_absolute_error:0.09993
[49]	validation_0-rmsle:0.09247	validation_0-mean_absolute_error:0.09951
[50]	validation_0-rmsle:0.09227	validation_0-mean_absolute_error:0.09907
[51]	validation_0-rmsle:0.09209	validation_0-mean_absolute_error:0.09868
[52]	validation_0-rmsle:0.09193	validation_0-mean_a

[52]	validation_0-rmsle:0.09195	validation_0-mean_absolute_error:0.09835
[53]	validation_0-rmsle:0.09180	validation_0-mean_absolute_error:0.09801
[54]	validation_0-rmsle:0.09165	validation_0-mean_absolute_error:0.09767
[55]	validation_0-rmsle:0.09152	validation_0-mean_absolute_error:0.09735
[56]	validation_0-rmsle:0.09140	validation_0-mean_absolute_error:0.09708
[57]	validation_0-rmsle:0.09129	validation_0-mean_absolute_error:0.09681
[58]	validation_0-rmsle:0.09118	validation_0-mean_absolute_error:0.09656
[59]	validation_0-rmsle:0.09108	validation_0-mean_absolute_error:0.09629
[60]	validation_0-rmsle:0.09097	validation_0-mean_absolute_error:0.09605
[61]	validation_0-rmsle:0.09088	validation_0-mean_absolute_error:0.09583
[62]	validation_0-rmsle:0.09079	validation_0-mean_absolute_error:0.09561
[63]	validation_0-rmsle:0.09071	validation_0-mean_absolute_error:0.09543
[64]	validation_0-rmsle:0.09064	validation_0-mean_absolute_error:0.09525
[65]	validation_0-rmsle:0.09057	validation_0-mean_a

In [30]:
np.mean(scores)

0.09019586857889558